# Examples of WaporAPI module functions

## 0. Change working directory to "wapor/src"

In [1]:
import os
import glob

p_from = os.getcwd()
p_root = os.path.join(os.getcwd(), '../', 'src')
os.chdir(p_root)
print('From "{c}" to "{t}"'.format(c=p_from, t=p_root))
print(os.listdir(p_root))

From "/notebooks/notebooks" to "/notebooks/notebooks/../src"
['wapor', 'WaPOR.egg-info']


## 1. First import the module

In [2]:
# from wapor.WaporAPI import __wapor_API_class
# API = __wapor_API_class()
# API.Token=input('Insert WAPOR API Token: ')

import wapor

Insert WAPOR API Token: ae34c8743c4dc4b3c32d26501fcef18b0cc47464baaa87cceb1b10d5ee1096ba03ab36196d29fe07


## 2. Get Catalog

In [3]:
print(wapor.API.Token)

wapor.API.version=2
catalog=wapor.API.getCatalog()

catalog

ae34c8743c4dc4b3c32d26501fcef18b0cc47464baaa87cceb1b10d5ee1096ba03ab36196d29fe07
Loading WaPOR catalog...


,code,caption,description,additionalInfo,tags,workspaceCode,dataType,index,operation,hidden,links,measure,dimension
0,L1_GBWP_A,Gross Biomass Water Productivity,The annual Gross Biomass Water Productivity ex...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 1, 'context': 'LEVELS...",WAPOR_2,RASTER,101,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'workspac..."
1,L1_NBWP_A,Net Biomass Water Productivity,The annual Net Biomass Water Productivity expr...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 2, 'context': 'LEVELS...",WAPOR_2,RASTER,102,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'workspac..."
2,L1_AETI_A,Actual EvapoTranspiration and Interception (An...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 3, 'context': 'LEVELS...",WAPOR_2,RASTER,103,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'YEAR', 'caption': 'Year', 'workspac..."
3,L1_AETI_M,Actual EvapoTranspiration and Interception (Mo...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 4, 'context': 'LEVELS...",WAPOR_2,RASTER,104,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'MONTH', 'caption': 'Month', 'worksp..."
4,L1_AETI_D,Actual EvapoTranspiration and Interception (De...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 5, 'context': 'LEVELS...",WAPOR_2,RASTER,105,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,L2_BAS_PHE_S,"Phenology (Seasonal, clipped by basin)","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4151,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'PHE', 'caption': 'Phenology', 'descr...","[{'code': 'YEAR', 'caption': 'Year', 'workspac..."
119,L2_BAS_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4161,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
120,L2_CTY_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4162,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
121,L2_BAS_QUAL_LST_D,"Quality Land Surface Temperature (Dekadal, cli...","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4171,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."


## 3. Get information of a cube

In [4]:
cube_code='L1_PCP_M'
cube_info=wapor.API.getCubeInfo(cube_code)
cube_info

{'code': 'L1_PCP_M',
 'caption': 'Precipitation (Monthly)',
 'description': 'Precipitation data is delivered on a daily basis. The source of this dataset is CHIRPS (Climate Hazards Group InfraRed Precipitation with Station) quasi-global rainfall dataset, starting from 1981 up to near present. The value of each pixel represents the total of daily precipitation in the month expressed in mm (1mm=1l/m² or 1mm=10m³/ha).',
 'additionalInfo': {'format': 'Raster Dataset',
  'unit': 'mm (1mm=1l/m² or 1mm=10m³/ha)',
  'dataType': 'Int16 (16bit Integer)',
  'conversionFactor': 'the pixel value in the downloaded data must be multiplied by 0.1',
  'noDataValue': -9999,
  'spatialResolution': 'Approximately 5km (0.05 degree)',
  'spatialExtent': 'Africa and Near East',
  'spatialReferenceSystem': 'EPSG:4326 - WGS84 - Geographic Coordinate System (lat/long)',
  'temporalResolution': 'from January 2009 to present',
  'temporalExtent': 'Monthly',
  'nearRealTime': 'New dekadal data layers are released 

Some useful information about dataset can be found in cube_info

In [5]:
print('Description: {0} \n\nmultiplier: {1} \n\nunit: {2} \n'.format(cube_info['description'],
                    cube_info['measure']['multiplier'],
                    cube_info['measure']['unit']))


Description: Precipitation data is delivered on a daily basis. The source of this dataset is CHIRPS (Climate Hazards Group InfraRed Precipitation with Station) quasi-global rainfall dataset, starting from 1981 up to near present. The value of each pixel represents the total of daily precipitation in the month expressed in mm (1mm=1l/m² or 1mm=10m³/ha). 

multiplier: 0.1 

unit: mm 



## 4. Get list of locations

In [6]:
df_locations=wapor.API.getLocations()
df_locations

,code,name,type,l1,l2,l3,bbox
0,DZA,Algeria,COUNTRY,False,False,False,"-8.66666793823242,18.963285446167,11.998500823..."
1,AGO,Angola,COUNTRY,False,False,False,"11.6792192459106,-18.0388412475586,24.08588981..."
2,7010,Awash,BASIN,False,True,False,"37.9583333333327,7.89583333333234,43.320833333..."
3,AWA,"Awash, Ethiopia",AREA,False,False,True,"518055.000, 912285.000, 613485.000, 988515.000"
4,BHR,Bahrain,COUNTRY,False,False,False,"50.3858451843262,25.5422401428223,50.828697204..."
...,...,...,...,...,...,...,...
74,ESH,Western Sahara,COUNTRY,False,False,False,"-17.1031837463379,20.7741584777832,-8.66664886..."
75,YEM,Yemen,COUNTRY,False,True,False,"41.8160438537598,12.1110925674438,54.530540466..."
76,ZMB,Zambia,COUNTRY,False,False,False,"21.9992122650146,-18.0794715881348,33.70570373..."
77,ZAN,"Zankalon, Egypt",AREA,False,False,True,"317985.000, 3348885.000, 377685.000, 3411885.000"


## 5. Get list of available data

In [7]:
df_avail=wapor.API.getAvailData(cube_code,
                                time_range='2009-01-01,2012-12-31')
df_avail

,MONTH,raster_id,bbox,time_code
0,2009-01,L1_PCP_0901M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-01-01,2009-02-01)"
1,2009-02,L1_PCP_0902M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-02-01,2009-03-01)"
2,2009-03,L1_PCP_0903M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-03-01,2009-04-01)"
3,2009-04,L1_PCP_0904M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-04-01,2009-05-01)"
4,2009-05,L1_PCP_0905M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-05-01,2009-06-01)"
5,2009-06,L1_PCP_0906M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-06-01,2009-07-01)"
6,2009-07,L1_PCP_0907M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-07-01,2009-08-01)"
7,2009-08,L1_PCP_0908M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-08-01,2009-09-01)"
8,2009-09,L1_PCP_0909M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-09-01,2009-10-01)"
9,2009-10,L1_PCP_0910M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-10-01,2009-11-01)"


## 6. Get download url of a raster
To get download url, wapor API Token is needed

In [8]:
APIToken=input('Enter your API Token: ')
wapor.API.Token=APIToken
wapor.API.getRasterUrl(cube_code,'L1_PCP_0901M',APIToken)

Enter your API Token: ae34c8743c4dc4b3c32d26501fcef18b0cc47464baaa87cceb1b10d5ee1096ba03ab36196d29fe07


{'url': 'https://io.apps.fao.org/gismgr/download/0732ab22-1c82-4612-a21e-f228e5e63169/L1_PCP_0901M.tif',
 'expiry_datetime': datetime.datetime(2019, 10, 17, 16, 52, 49, 708293)}

## 7. Get download url of a cropped raster
To request a cropped raster, raster_id and time_code of the raster is needed

In [9]:
bbox= [37.95883206252312, 7.89534, 43.32093, 12.3873979377346]
rasterId='L1_PCP_0901M'
dimension_value='[2009-01-01,2009-02-01)'
wapor.API.getCropRasterURL(bbox,cube_code,
                          dimension_value,rasterId,APIToken)

Getting download url from: https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/a72d48d7-1767-4795-bca4-e87a80736fd0


'https://io.apps.fao.org/gismgr/download/86b9ca18-ff8f-432b-a4d5-744f05ced507/L1_PCP_0901M.tif'

## 8. Get Area time-series
A shapefile can be used to define the Region of Interest

In [10]:
Dir = "Shapefile"
chk_files=glob.glob(os.path.join(Dir, '*.shp'))
chk_files

[]

In [11]:
shp_fh=os.path.join(Dir, "Awash_HydroSHED.shp")
ts_area=wapor.API.getAreaTimeseries(shp_fh,'L1_PCP_M',APIToken,
                                       time_range="2009-01-01,2018-12-31")
ts_area

AttributeError: 'NoneType' object has no attribute 'GetLayer'

## 9. Get Point time-series

In [ ]:
Point=[37.95883206252312, 7.89534] #longitude and latitude
ts_point=wapor.API.getPixelTimeseries(Point,'L1_PCP_M',
                                      time_range="2009-01-01,2018-12-31")
ts_point

## 10. Bulk-downloading

In [ ]:
import requests
Dir='Data'
chk_files=glob.glob(os.path.join(Dir, '*.tif'))
chk_files

In [ ]:
for index,row in df_avail.iterrows():   
    download_url=wapor.API.getCropRasterURL(bbox,cube_code,
                                           row['time_code'],
                                           row['raster_id'],
                                           wapor.API.Token,
                                           print_job=False)
    download_file=os.path.join(Dir,'{0}.tif'.format(row['raster_id']))
    resp=requests.get(download_url) 
    open(download_file,'wb').write(resp.content) 

In [ ]:
chk_files=glob.glob(os.path.join(Dir, '*.tif'))
chk_files